In [33]:
from newsapi import NewsApiClient
import pandas as pd
from datetime import date, timedelta
from newspaper import Article, Config
import nltk
from nltk.corpus import stopwords

today = date.today()
yesterday = today - timedelta(days = 1)

# Init
newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(   
                                          category='entertainment',
                                          language='en',
                                          page_size = 90,
                                          page= 1)

articles = top_headlines.get('articles',[])

init_df = pd.DataFrame(articles, columns = ['source','title','publishedAt','author','url'])

init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])

yesterday_data = init_df[init_df['publishedAt'].dt.date == yesterday]


df = yesterday_data.copy()

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config = config)

    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'


df['content'] = df['url'].apply(full_content)
df['content'] = df['content'].str.replace('\n', ' ')
df = df[df['content'] != 'couldnt retrieve']



# Download the stopwords dataset
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


def count_words_without_stopwords(text):
    if isinstance(text, (str, bytes)):
        words = nltk.word_tokenize(str(text))
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return len(filtered_words)
    else:
        0

df['word_count'] = df['content'].apply(count_words_without_stopwords)
df


TypeError: unsupported operand type(s) for |: 'str' and 'bool'

In [31]:
filtered_df

,source,title,publishedAt,author,url
